In [ ]:
!unzip -e /content/drive/My\ Drive/Ажил/3405_6663_bundle_archive.zip

Archive:  /content/drive/My Drive/Ажил/3405_6663_bundle_archive.zip
  inflating: credits.csv             
  inflating: keywords.csv            
  inflating: links.csv               
  inflating: links_small.csv         
  inflating: movies_metadata.csv     
  inflating: ratings.csv             
  inflating: ratings_small.csv       


In [1]:
import pandas as pd

metadata = pd.read_csv('./data/movies_metadata.csv', low_memory=False)

metadata.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


# Section 1

In [2]:
C = metadata['vote_average'].mean()
print(C)

5.618207215134185


In [3]:
m = metadata['vote_count'].quantile(0.90)
print(m)

160.0


In [4]:
q_movies = metadata.copy().loc[metadata['vote_count'] >= m]
q_movies.shape

(4555, 24)

In [5]:
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [6]:
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)


In [7]:
q_movies = q_movies.sort_values('score', ascending=False)

q_movies[['title', 'vote_count', 'vote_average', 'score']].head(20)

,title,vote_count,vote_average,score
314,The Shawshank Redemption,8358.0,8.5,8.445869
834,The Godfather,6024.0,8.5,8.425439
10309,Dilwale Dulhania Le Jayenge,661.0,9.1,8.421453
12481,The Dark Knight,12269.0,8.3,8.265477
2843,Fight Club,9678.0,8.3,8.256385
292,Pulp Fiction,8670.0,8.3,8.251406
522,Schindler's List,4436.0,8.3,8.206639
23673,Whiplash,4376.0,8.3,8.205404
5481,Spirited Away,3968.0,8.3,8.196055
2211,Life Is Beautiful,3643.0,8.3,8.187171


In [8]:
metadata['overview'].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')


metadata['overview'] = metadata['overview'].fillna('')

tfidf_matrix = tfidf.fit_transform(metadata['overview'])

tfidf_matrix.shape

(45466, 75827)

In [10]:
tfidf.get_feature_names()[5000:5010]


['avails',
 'avaks',
 'avalanche',
 'avalanches',
 'avallone',
 'avalon',
 'avant',
 'avanthika',
 'avanti',
 'avaracious']

In [13]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

ValueError: array is too big; `arr.size * arr.dtype.itemsize` is larger than the maximum possible size.

ValueError: array is too big; `arr.size * arr.dtype.itemsize` is larger than the maximum possible size.

In [6]:
credits = pd.read_csv('./data/credits.csv')
keywords = pd.read_csv('./data/keywords.csv')

# Remove rows with bad IDs.
metadata = metadata.drop([19730, 29503, 35587])

# Convert IDs to int. Required for merging
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
metadata['id'] = metadata['id'].astype('int')

# Merge keywords and credits into your main metadata dataframe
metadata = metadata.merge(credits, on='id')
metadata = metadata.merge(keywords, on='id')

NameError: name 'metadata' is not defined

In [ ]:
metadata.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."


In [ ]:
from ast import literal_eval

features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(literal_eval)

In [ ]:
import numpy as np
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

In [ ]:
metadata['director'] = metadata['crew'].apply(get_director)

features = ['cast', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(get_list)
    

In [ ]:
metadata[['title', 'cast', 'director', 'keywords', 'genres']].head(3)


,title,cast,director,keywords,genres
0,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles]",John Lasseter,"[jealousy, toy, boy]","[Animation, Comedy, Family]"
1,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]",Joe Johnston,"[board game, disappearance, based on children'...","[Adventure, Fantasy, Family]"
2,Grumpier Old Men,"[Walter Matthau, Jack Lemmon, Ann-Margret]",Howard Deutch,"[fishing, best friend, duringcreditsstinger]","[Romance, Comedy]"


# section 2


In [ ]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace("]", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace("]", ""))
        else:
            return ''

In [ ]:
features = ['categories', 'subCategories', 'tags']

for feature in features:
    metadata[feature] = metadata[feature].apply(clean_data)

In [ ]:
metadata[feature] = metadata['tags'].apply(clean_data)

In [ ]:
metadata.head()

,categories,subCategories,tags,soup
0,arts entertainment,art,"[ball rhythmic gymnastics, bead, carving, clow...",[ b a l l r h y t h m i c g y m n a s t i ...
1,arts entertainment,artwork,"[anime, bouquet, cartoon, drawing, graphics, m...","[ a n i m e , b o u q u e t , c a r t o o ..."
2,arts entertainment,entertainment,"[ball rhythmic gymnastics, clown, comedy, danc...",[ b a l l r h y t h m i c g y m n a s t i ...
3,astronomical object,comet,[],[ ] comet astronomical object
4,astronomical object,galaxy,[milky way],[ m i l k y w a y ] galaxy astronomical object


In [ ]:
def create_soup(x):
    return x['subCategories'] + ' ' + ''.join(x['tags']) 

In [ ]:
metadata['soup'] = metadata.apply(create_soup, axis=1)


In [ ]:
metadata[['soup']].head(6)


,soup
0,"art [ball rhythmic gymnastics, bead, carving, ..."
1,"artwork [anime, bouquet, cartoon, drawing, gra..."
2,"entertainment [ball rhythmic gymnastics, clown..."
3,comet []
4,galaxy [milky way]
5,moon []


In [ ]:
metadata[feature] = metadata['soup'].apply(clean_data)
metadata.head(9)

,categories,subCategories,tags,soup
0,arts entertainment,art,"art [ball rhythmic gymnastics, bead, carving, ...","art [ball rhythmic gymnastics, bead, carving, ..."
1,arts entertainment,artwork,"artwork [anime, bouquet, cartoon, drawing, gra...","artwork [anime, bouquet, cartoon, drawing, gra..."
2,arts entertainment,entertainment,"entertainment [ball rhythmic gymnastics, clown...","entertainment [ball rhythmic gymnastics, clown..."
3,astronomical object,comet,comet [,comet []
4,astronomical object,galaxy,galaxy [milky way,galaxy [milky way]
5,astronomical object,moon,moon [,moon []
6,astronomical object,nebula,nebula [,nebula []
7,astronomical object,orbit,orbit [,orbit []
8,astronomical object,outer space,outer space [,outer space []


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(metadata['soup'])

In [ ]:
count_matrix.shape

(802, 27)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # Гарчигтай тохирч байгаа киноны индексийг авах
    idx = indices[title]

    # Тэр киногоороо бүх киноны хослолын ижил төстэй оноог аваарай
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Ижил төстэй оноог үндэслэн кинонуудыг эрэмбэлэх
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:11]

    # Индексыг авах
    movie_indices = [i[0] for i in sim_scores]

    # Хамгийн төстэй 10 киноны шилдэг 10-г буцааж өг
    return metadata['categories'].iloc[movie_indices]

In [ ]:
metadata = metadata.reset_index()
indices = pd.Series(metadata.index, index=metadata['categories'])

In [ ]:
get_recommendations('drink', cosine_sim)

ValueError: ignored